In [2]:
import pandas as panda

from sklearn import metrics
from sklearn.model_selection import train_test_split




from google.colab import drive
drive.mount('/content/drive')

df = panda.read_csv('/content/drive/MyDrive/feature_complete1.3manhattenyellowtaxis.csv')



print(df.head())







Mounted at /content/drive
   trip_distance  fare_amount  total_amount  extra  tip_amount  \
0           3.15         15.0     19.560000    0.5    3.260000   
1           0.83          5.5      6.800000    0.5    0.000000   
2           1.00          5.0      6.300000    0.5    0.000000   
3           2.32         11.0     12.786667    0.5    0.486667   
4           1.46          7.5     11.440000    0.5    2.640000   

   congestion_surcharge  ride_duration  average_speed  DOLocation_area  \
0                     1      21.150000       8.936170         0.000114   
1                     1       5.550000       8.972973         0.000438   
2                     1       3.816667      15.720524         0.000143   
3                     1      13.788889       9.586343         0.000094   
4                     1       7.850000      11.159236         0.000111   

   DOLocation_perimeter  ...  dropoff_month_3  dropoff_month_4  \
0              0.047584  ...                0                0   


In [3]:
from xgboost import XGBClassifier
from sklearn import metrics

training_size =  int(0.8 *   len(df))

training_df =   df.iloc[:training_size]
testing_df =  df.iloc[training_size:]

X_training =   training_df.drop('busyness',  axis=1)
y_training =  training_df['busyness']




X_testing =   testing_df.drop('busyness',  axis=1)
y_testing =  testing_df['busyness']

y_training = y_training - 1
y_testing = y_testing - 1

model = XGBClassifier(tree_method='gpu_hist',   random_state=40,
  use_label_encoder=False,   eval_metric='mlogloss')


model.fit(X_training, y_training)


y_predict = model.predict(X_testing)

# make the metrics
accuracy =  metrics.accuracy_score(y_testing, y_predict)
f1_score =    metrics.f1_score(y_testing, y_predict, average='macro')
recall =   metrics.recall_score(y_testing, y_predict, average='macro')
precision =   metrics.precision_score(y_testing, y_predict, average='macro')

# Print the metrics
print("accuracy:",  accuracy)
print("f1:",   f1_score)
print("recall:",   recall)
print("precision:",   precision)






/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


accuracy: 0.832203512061443
f1: 0.8312046163144415
recall: 0.8318305691870584
precision: 0.8314284671885155


In [4]:
import pickle



with open('/content/drive/My Drive/t10XGBclassifier.pkl',  'wb') as file:
    pickle.dump(model, file)






In [5]:
i = model.feature_importances_



how_important_df = panda.DataFrame({
    'Feature': X_training.columns,
    'Importance': i
})


how_important_df.sort_values('Importance', ascending=False, inplace=True)


how_important_df.to_csv('/content/drive/My Drive/t10.3feature_importances.csv',  index=False)
